In [ ]:
import pandas as pd
data = pd.read_csv("processed_data.csv")
# Extract datetime components and join them with hyphens
datetime_str = (data["Serial"].astype(str).str[:4] + '-' +    # Year
                   data["Serial"].astype(str).str[4:6] + '-' +    # Month
                   data["Serial"].astype(str).str[6:8] + '-' +    # Day
                   data["Serial"].astype(str).str[8:12])          # Hour and Minute
# Convert to datetime
data['Datetime'] = pd.to_datetime(datetime_str, format='%Y-%m-%d-%H%M')
data["DeviceID"] = data["Serial"].astype(str).str[12:14].astype(int)
data['Type'] = data['DeviceID'].apply(lambda x: 1 if 1 <= x <= 14 else 2)
# 只保留時間介於 6:00 ~ 18:00 的資料
# data = data[(data['Datetime'].dt.hour >= 6) & (data['Datetime'].dt.hour < 18)]

data['day_of_year'] = [i.dayofyear for i in data['Datetime']]
data['month'] = [i.month for i in data['Datetime']]
data['day'] =  [i.day for i in data['Datetime']]
data['hour'] = [i.hour for i in data['Datetime']]
data['minute'] = [i.minute for i in data['Datetime']]
data["hhmm"] = data["Serial"].astype(str).str[8:12].astype(int)

In [ ]:
from openWeather import openWeather
data, weather_columns = openWeather(data)

In [ ]:
# Filter data to get the rows where the time is 08:50
data_850 = data[(data['hour'] == 8) & (data['minute'] == 50)]

# Select only the required columns
data_850 = data_850[['DeviceID', 'day_of_year', 'Pressure(hpa)', 'WindSpeed(m/s)', 'Temperature(°C)', 'Sunlight(Lux)', 'Humidity(%)']]

# Rename columns to indicate they are from 08:50
data_850.columns = ['DeviceID', 'day_of_year', 'Pressure_850', 'WindSpeed_850', 'Temperature_850', 'Sunlight_850', 'Humidity_850']

# Merge the original data with the 08:50 data
data = pd.merge(data, data_850, on=['DeviceID', 'day_of_year'], how='left', suffixes=('', '_duplicate'))

# Drop duplicate columns
data.drop(columns=["Pressure(hpa)", "WindSpeed(m/s)", "Temperature(°C)", "Sunlight(Lux)", "Humidity(%)"], inplace=True)
data

In [ ]:
import joblib
humidity_model = "humidity_model.joblib"
pressure_model = "pressure_model.joblib"
sunlight_model = "sunlight_model.joblib"
temperature_model = "temperature_model.joblib"
wind_speed_model = "wind_speed_model.joblib"

X = data[
        [
            "hour",
            "minute",
            "DeviceID",
            *weather_columns,
            "Pressure_850",
            "WindSpeed_850",
            "Temperature_850",
            "Sunlight_850",
            "Humidity_850",
        ]
    ]

data["Pressure(hpa)"] = joblib.load(pressure_model).predict(X)
data["WindSpeed(m/s)"] = joblib.load(wind_speed_model).predict(X)
data["Temperature(°C)"] = joblib.load(temperature_model).predict(X)
data["Sunlight(Lux)"] = joblib.load(sunlight_model).predict(X)
data["Humidity(%)"] = joblib.load(humidity_model).predict(X)


In [ ]:
import numpy as np

def encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

data['month'] = data['Datetime'].dt.month
data = encode(data, 'month', 12)

# data['day'] = data['Datetime'].dt.day
# data = encode(data, 'day', 31)
data.drop(columns=["month", "day"], inplace=True)

In [ ]:
data.columns

In [ ]:
from tqdm import tqdm
from pycaret.regression import *

all_Device = data['DeviceID'].unique()
all_results = []
final_model = {}

for i in tqdm(all_Device):
    df_subset = data[data['DeviceID'] == i]
    
    s = setup(df_subset, 
              use_gpu = False, 
              target = 'Power(mW)',
              train_size = 0.95,
              data_split_shuffle = False,
              ignore_features = ['Datetime', 'DeviceID', 'Serial', 'yyyymmddhh'],
              fold_strategy = "timeseries",
              fold = 5,
              verbose = False, session_id = 123, normalize = True
             )
    # compare all models and select best one based on MAE
    best_model = compare_models(sort = 'MAE', verbose = True, exclude = ['lightgbm'])
    
    # capture the compare result grid and store best model in list
    p = pull().iloc[0:1]
    p['time_series'] = str(i)
    all_results.append(p)
    
    # finalize model i.e. fit on entire data including test set
    f = finalize_model(best_model)
    
    # attach final model to a dictionary
    final_model[i] = f
    
    # save transformation pipeline and model as pickle file 
    save_model(f, model_name='trained_models/' + str(i), verbose=False)

In [ ]:
concat_results = pd.concat(all_results,axis=0)
concat_results

In [1]:
import pandas as pd
from openWeather import openWeather

upload_data = pd.read_csv('upload(no answer).csv')
# rename columns
upload_data.columns = ["Serial", "Power(mW)"]
# Extract datetime components and join them with hyphens
datetime_str = (upload_data["Serial"].astype(str).str[:4] + '-' +    # Year
                   upload_data["Serial"].astype(str).str[4:6] + '-' +    # Month
                   upload_data["Serial"].astype(str).str[6:8] + '-' +    # Day
                   upload_data["Serial"].astype(str).str[8:12])          # Hour and Minute
# Convert to datetime
upload_data['Datetime'] = pd.to_datetime(datetime_str, format='%Y-%m-%d-%H%M')
upload_data["DeviceID"] = upload_data["Serial"].astype(str).str[12:14].astype(int)
upload_data['Type'] = upload_data['DeviceID'].apply(lambda x: 1 if 1 <= x <= 14 else 2)

upload_data['day_of_year'] = [i.dayofyear for i in upload_data['Datetime']]
upload_data['month'] = [i.month for i in upload_data['Datetime']]
upload_data['day'] =  [i.day for i in upload_data['Datetime']]
upload_data['hour'] = [i.hour for i in upload_data['Datetime']]
upload_data['minute'] = [i.minute for i in upload_data['Datetime']]
upload_data["hhmm"] = upload_data["Serial"].astype(str).str[8:12].astype(int)

# weather data
upload_data, weather_columns = openWeather(upload_data)
upload_data

Index(['PS01_C0Z100', 'TX01_C0Z100', 'RH01_C0Z100', 'WD01_C0Z100',
       'WD02_C0Z100', 'WD07_C0Z100', 'WD08_C0Z100', 'PP01_C0Z100'],
      dtype='object')


,Serial,Power(mW),Datetime,DeviceID,Type,day_of_year,month,day,hour,minute,...,WS_Flag_鳳林生豐站,WD_鳳林生豐站,WD_Flag_鳳林生豐站,Ts_鳳林生豐站,Ts_Flag_鳳林生豐站,SWC_鳳林生豐站,SWC_Flag_鳳林生豐站,CO2_Flag_鳳林生豐站,H_Flag_鳳林生豐站,LE_Flag_鳳林生豐站
96,20240117090001,NaN,2024-01-17 09:00:00,1,1,17,1,17,9,0,...,0.0,182.460,0.0,16.986,0.0,33.3833,0.0,1.0,0.0,1.0
97,20240117091001,NaN,2024-01-17 09:10:00,1,1,17,1,17,9,10,...,0.0,182.460,0.0,16.986,0.0,33.3833,0.0,1.0,0.0,1.0
98,20240117092001,NaN,2024-01-17 09:20:00,1,1,17,1,17,9,20,...,0.0,189.056,0.0,17.065,0.0,33.3905,0.0,2.0,0.0,2.0
99,20240117093001,NaN,2024-01-17 09:30:00,1,1,17,1,17,9,30,...,0.0,189.056,0.0,17.065,0.0,33.3905,0.0,2.0,0.0,2.0
100,20240117094001,NaN,2024-01-17 09:40:00,1,1,17,1,17,9,40,...,0.0,189.056,0.0,17.065,0.0,33.3905,0.0,2.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9307,20241002161012,NaN,2024-10-02 16:10:00,12,1,276,10,2,16,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9308,20241002162012,NaN,2024-10-02 16:20:00,12,1,276,10,2,16,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9309,20241002163012,NaN,2024-10-02 16:30:00,12,1,276,10,2,16,30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9310,20241002164012,NaN,2024-10-02 16:40:00,12,1,276,10,2,16,40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# 打開參考資料
SourceData = pd.read_csv("processed_data.csv")
SourceData, weather_columns = openWeather(SourceData)
SourceData["Datetime"] = pd.to_datetime(
    SourceData["Serial"].astype(str).str[:12], format="%Y%m%d%H%M"
)
SourceData["DeviceID"] = SourceData["Serial"].astype(str).str[12:14].astype(int)
SourceData["day_of_year"] = [i.dayofyear for i in SourceData["Datetime"]]
SourceData["hour"] = [i.hour for i in SourceData["Datetime"]]
SourceData["minute"] = [i.minute for i in SourceData["Datetime"]]
# Filter data to get the rows where the time is 08:50
data_850 = SourceData[(SourceData["hour"] == 8) & (SourceData["minute"] == 50)]

# Select only the required columns
data_850 = data_850[
    [
        "DeviceID",
        "day_of_year",
        "Pressure(hpa)",
        "WindSpeed(m/s)",
        "Temperature(°C)",
        "Sunlight(Lux)",
        "Humidity(%)",
    ]
]

# Rename columns to indicate they are from 08:50
data_850.columns = [
    "DeviceID",
    "day_of_year",
    "Pressure_850",
    "WindSpeed_850",
    "Temperature_850",
    "Sunlight_850",
    "Humidity_850",
]

upload_data = pd.merge(
    upload_data,
    data_850,
    on=["DeviceID", "day_of_year"],
    how="left",
    suffixes=("", "_duplicate"),
)

Index(['PS01_C0Z100', 'TX01_C0Z100', 'RH01_C0Z100', 'WD01_C0Z100',
       'WD02_C0Z100', 'WD07_C0Z100', 'WD08_C0Z100', 'PP01_C0Z100'],
      dtype='object')


In [3]:
import joblib
humidity_model = "humidity_model.joblib"
pressure_model = "pressure_model.joblib"
sunlight_model = "sunlight_model.joblib"
temperature_model = "temperature_model.joblib"
wind_speed_model = "wind_speed_model.joblib"

X = upload_data[
    [
        "hour",
        "minute",
        "DeviceID",
        *weather_columns,
        "Pressure_850",
        "WindSpeed_850",
        "Temperature_850",
        "Sunlight_850",
        "Humidity_850",
    ]
]

upload_data["Pressure(hpa)"] = joblib.load(pressure_model).predict(X)
upload_data["WindSpeed(m/s)"] = joblib.load(wind_speed_model).predict(X)
upload_data["Temperature(°C)"] = joblib.load(temperature_model).predict(X)
upload_data["Sunlight(Lux)"] = joblib.load(sunlight_model).predict(X)
upload_data["Humidity(%)"] = joblib.load(humidity_model).predict(X)

In [4]:
import numpy as np

def encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

upload_data['month'] = upload_data['Datetime'].dt.month
upload_data = encode(upload_data, 'month', 12)

# data['day'] = data['Datetime'].dt.day
# data = encode(data, 'day', 31)
upload_data.drop(columns=["month", "day"], inplace=True)

In [6]:
from tqdm import tqdm
from pycaret.regression import load_model, predict_model
all_Device = upload_data['DeviceID'].unique()
all_score_df = []

for i in tqdm(all_Device):
    df_subset = upload_data[upload_data['DeviceID'] == i]
    df_subset = df_subset.drop(columns=["Power(mW)"])
    l = load_model('trained_models/' + str(i), verbose=False)
    p = predict_model(l, data=df_subset)
    p['DeviceID'] = i
    all_score_df.append(p)

concat_df = pd.concat(all_score_df, axis=0)
concat_df.head()

100%|██████████| 17/17 [00:05<00:00,  2.93it/s]


,Serial,Datetime,DeviceID,Type,day_of_year,hour,minute,hhmm,yyyymmddhh,PS01_C0Z100,...,Sunlight_850,Humidity_850,Pressure(hpa),WindSpeed(m/s),Temperature(°C),Sunlight(Lux),Humidity(%),month_sin,month_cos,prediction_label
0,20240117090001,2024-01-17 09:00:00,1,1,17,9,0,900,2024011709,NaN,...,20943.169922,100.0,1014.890320,0.595301,20.928242,19241.083984,97.664009,0.5,0.866025,89.703099
1,20240117091001,2024-01-17 09:10:00,1,1,17,9,10,910,2024011709,NaN,...,20943.169922,100.0,1014.883301,0.593254,21.432674,20886.824219,96.213249,0.5,0.866025,117.035100
2,20240117092001,2024-01-17 09:20:00,1,1,17,9,20,920,2024011709,NaN,...,20943.169922,100.0,1017.393005,0.509767,21.577967,21286.353516,96.592651,0.5,0.866025,109.526801
3,20240117093001,2024-01-17 09:30:00,1,1,17,9,30,930,2024011709,NaN,...,20943.169922,100.0,1017.921814,0.508208,21.837189,24639.152344,95.871498,0.5,0.866025,159.461900
4,20240117094001,2024-01-17 09:40:00,1,1,17,9,40,940,2024011709,NaN,...,20943.169922,100.0,1015.586182,0.507348,22.395863,26097.015625,94.647408,0.5,0.866025,183.764000


In [7]:
concat_df["prediction_label"] = np.maximum(concat_df["prediction_label"], 0)
concat_df["prediction_label"] = np.round(concat_df["prediction_label"], 2)
output = pd.DataFrame({"序號": concat_df["Serial"], "答案": concat_df["prediction_label"]})
output.to_csv("predictions.csv", index=False, encoding="utf-8", header=False)

In [8]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = concat_df["prediction_label"]
processed_data = pd.read_csv("processed_data.csv")
y_test = processed_data.loc[processed_data["Serial"].isin(concat_df["Serial"])][
    "Power(mW)"
]
# Handle NaN values
y_test = y_test.fillna(0)
y_pred = np.nan_to_num(y_pred)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")

score = sum(abs(y_test - y_pred))
print(f"Score: {score}")


ValueError: Found input variables with inconsistent numbers of samples: [0, 9600]